In [ ]:
import yfinance as yf
# 下載全球指數資料
indices = '^GSPC ^DJI ^FTSE ^N225'  # S&P 500, Dow Jones, FTSE 100, Nikkei 225
index_data = yf.download(indices, start='2022-01-01', end='2024-01-01')

# 顯示指數數據
print(index_data)

In [2]:
import yfinance as yf

# 下載不同國家的股票資料
# 美股: 蘋果 (AAPL) 和 微軟 (MSFT)
# 台股: 台積電 (2330.TW) 和 鴻海 (2317.TW)
# 港股: 匯豐控股 (0005.HK) 和 中國平安 (2318.HK)
tickers = 'AAPL MSFT 2330.TW 2317.TW 0005.HK 2318.HK'

# 下載資料
data = yf.download(tickers, start='2022-01-01', end='2024-01-01')

# 顯示數據
print(data)

[*********************100%***********************]  6 of 6 completed


Price                      Adj Close                                     \
Ticker                       0005.HK     2317.TW    2318.HK     2330.TW   
Date                                                                      
2022-01-03 00:00:00+00:00  38.732952   91.097260  46.399651  596.844116   
2022-01-04 00:00:00+00:00  39.518360   91.539482  47.023853  620.490967   
2022-01-05 00:00:00+00:00  40.634464   96.403900  47.273544  614.815735   
2022-01-06 00:00:00+00:00  40.510452   94.635017  48.272282  609.140381   
2022-01-07 00:00:00+00:00  41.419872   94.192795  50.852341  599.681763   
...                              ...         ...        ...         ...   
2023-12-25 00:00:00+00:00        NaN  100.912506        NaN  573.515564   
2023-12-26 00:00:00+00:00        NaN  101.400002        NaN  578.451111   
2023-12-27 00:00:00+00:00  57.857643  100.912506  31.104734  584.373840   
2023-12-28 00:00:00+00:00  58.412632  101.400002  32.876820  585.360901   
2023-12-29 00:00:00+00:00

In [ ]:
index_names = {
    '^GSPC': '標準普爾500指數(S&P 500)',
    '^DJI': '道瓊斯工業平均指數',
    '^IXIC': '納斯達克綜合指數',
    '^FTSE': 'FTSE 100 指數（英國）',
    '^N225': '日經225指數（日本）',
    '^TWII': '台灣加權指數'
}

# 顯示指數名稱及其代號
for ticker in indices:
    INSERT INTO 市場 values ('^TWII','台灣')
    ON CONFLICT DO NOTHING;
   #  print(f"{ticker}: {index_names[ticker]}")



In [ ]:
import streamlit as st
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

@st.cache_data
def getData(country:tuple[str])->list[tuple]:
    conn = psycopg2.connect(host=os.environ     ['POSTGRE_HOST'],
                            database=os.environ ['POSTGRE_DATABASE'],
                            user=os.environ     ['POSTGRE_USER'],
                            password=os.environ ['POSTGRE_PASSWORD'])

In [ ]:
import streamlit as st
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

conn = psycopg2.connect(host            = os.environ    ['POSTGRE_HOST'],
                        database        = os.environ    ['POSTGRE_DATABASE'],
                        user            = os.environ    ['POSTGRE_USER'],
                        password        = os.environ    ['POSTGRE_PASSWORD'])

sql1 =  '''
        INSERT INTO 市場 values (%s,%s)
        ON CONFLICT DO NOTHING;
        '''

sql2 =  '''
        CREATE TABLE IF NOT EXISTS 股市(
        stock_id SERIAL,
        date DATE,
        open NUMERIC(17,10),
        high NUMERIC(17,10),
        low NUMERIC(17,10),
        close NUMERIC(17,10),
        adj_close NUMERIC(17,10),
        volume BIGINT DEFAULT 0,
        name VARCHAR(20),
        PRIMARY KEY(stock_id),
        UNIQUE(date,name)
        FOREIGN KEY(name) REFERENCES 市場(name) 
        ON DELETE NO ACTION
        ON UPDATE CASCADE
        ) 
        ON CONFLICT DO NOTHING;
        '''

sql1 =  '''
        INSERT INTO 市場 values (%s,%s)
        ON CONFLICT DO NOTHING;
        '''
with conn:
    with conn.cursor() as cursor:        
        cursor.execute(sql1,['^TWII','台灣'])
#    with conn.cursor() as cursor:        
        cursor.execute(sql1,['^HSI','香港恆生'])

conn.close()

# 新增市場及代號對照資料

In [ ]:
import streamlit as st
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()

# 連接到 PostgreSQL 資料庫
conn = psycopg2.connect(host            = os.environ    ['POSTGRE_HOST'],
                        database        = os.environ    ['POSTGRE_DATABASE'],
                        user            = os.environ    ['POSTGRE_USER'],
                        password        = os.environ    ['POSTGRE_PASSWORD'])

# 定義指數代號與名稱的字典
index_names = {
    '^GSPC' : '標準普爾500指數(S&P 500)',
    '^DJI'  : '道瓊斯工業平均指數',
    '^IXIC' : '納斯達克綜合指數',
    '^FTSE' : 'FTSE 100 指數（英國）',
    '^N225' : '日經225指數(日本)',
    '^TWII' : '台灣加權指數',
    '^HSI'  : '香港恆生'
}

# SQL 插入語句
sql1 =  '''
        INSERT INTO 市場 VALUES (%s, %s)
        ON CONFLICT DO NOTHING;
        '''

# 使用上下文管理器確保連接正確關閉
with conn:
    with conn.cursor() as cursor:
        # 遍歷字典並執行插入操作
        for index_code, index_name in index_names.items():
            cursor.execute(sql1, [index_code, index_name])

# 關閉資料庫連接
conn.close()

In [10]:
import streamlit as st
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import yfinance as yf
from decimal import Decimal

# 連接到 PostgreSQL 資料庫
conn = psycopg2.connect(host            = os.environ    ['POSTGRE_HOST'],
                        database        = os.environ    ['POSTGRE_DATABASE'],
                        user            = os.environ    ['POSTGRE_USER'],
                        password        = os.environ    ['POSTGRE_PASSWORD'])

index_names = {
    '^GSPC' : '標準普爾500指數(S&P 500)',
    '^DJI'  : '道瓊斯工業平均指數',
    '^IXIC' : '納斯達克綜合指數',
    '^FTSE' : 'FTSE 100 指數（英國）',
    '^N225' : '日經225指數(日本)',
    '^TWII' : '台灣加權指數',
    '^HSI'  : '香港恆生'
}
c_name = '^IXIC'
country = yf.download(c_name)

sql4 =  '''
        INSERT INTO 股市(date,open,high,low,close,adj_close,volume,name) 
        values(%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT DO NOTHING;
        '''
with conn:
    all_data        = country.to_dict(orient='index')
    for key,value in all_data.items():
        date        = key.date()
        name        = c_name
        adj_close   = Decimal(value[('Adj Close', name)])
        open        = Decimal(value[('Open'     , name)])
        high        = Decimal(value[('High'     , name)])
        low         = Decimal(value[('Low'      , name)])
        close       = Decimal(value[('Close'    , name)])
        volume      = Decimal(value[('Volume'   , name)])
        with conn.cursor() as cursor:            
            cursor.execute(sql4,[date,open,high,low,close,adj_close,volume,name])

conn.close()
print(all_data)

[*********************100%***********************]  1 of 1 completed


{Timestamp('1971-02-05 00:00:00+0000', tz='UTC'): {('Adj Close', '^IXIC'): 100.0, ('Close', '^IXIC'): 100.0, ('High', '^IXIC'): 100.0, ('Low', '^IXIC'): 100.0, ('Open', '^IXIC'): 100.0, ('Volume', '^IXIC'): 0}, Timestamp('1971-02-08 00:00:00+0000', tz='UTC'): {('Adj Close', '^IXIC'): 100.83999633789062, ('Close', '^IXIC'): 100.83999633789062, ('High', '^IXIC'): 100.83999633789062, ('Low', '^IXIC'): 100.83999633789062, ('Open', '^IXIC'): 100.83999633789062, ('Volume', '^IXIC'): 0}, Timestamp('1971-02-09 00:00:00+0000', tz='UTC'): {('Adj Close', '^IXIC'): 100.76000213623047, ('Close', '^IXIC'): 100.76000213623047, ('High', '^IXIC'): 100.76000213623047, ('Low', '^IXIC'): 100.76000213623047, ('Open', '^IXIC'): 100.76000213623047, ('Volume', '^IXIC'): 0}, Timestamp('1971-02-10 00:00:00+0000', tz='UTC'): {('Adj Close', '^IXIC'): 100.69000244140625, ('Close', '^IXIC'): 100.69000244140625, ('High', '^IXIC'): 100.69000244140625, ('Low', '^IXIC'): 100.69000244140625, ('Open', '^IXIC'): 100.69000